# Data

Code used to arrange data into a format more convenient for the thesis. We calculate the required predictors from the data provided by Amit Goyal (https://sites.google.com/view/agoyal145/?redirpath=/), from the dataset used in his paper "A Comprehensive Look at the Empirical Performance of Equity Premium Prediction" (with Ivo Welch), July 2008. We use data updated to include monthly observations up to 2021.

### Imports and Initialization

In [1]:
# external imports
import pandas as pd
import numpy as np

In [2]:
# loading Goyal data
GoyalData_updated = pd.read_csv("GoyalData_2021.csv", thousands=',')
data = GoyalData_updated

# starting and ending periods
TRAIN_START = 192612
TRAIN_END = 195612
TEST_START = 195701
TEST_END = 201612

IND_TRAIN_START = data[data['yyyymm'] == TRAIN_START].index[0]
IND_TRAIN_END = data[data['yyyymm'] == TRAIN_END].index[0]
IND_TEST_START = data[data['yyyymm'] == TEST_START].index[0]
IND_TEST_END = data[data['yyyymm'] == TEST_END].index[0]

data

,yyyymm,Index,D12,E12,b/m,tbl,AAA,BAA,lty,ntis,Rfree,infl,ltr,corpr,svar,csp,CRSP_SPvw,CRSP_SPvwx
0,187101,4.44,0.2600,0.4000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,187102,4.50,0.2600,0.4000,NaN,NaN,NaN,NaN,NaN,NaN,0.004967,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,187103,4.61,0.2600,0.4000,NaN,NaN,NaN,NaN,NaN,NaN,0.004525,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,187104,4.74,0.2600,0.4000,NaN,NaN,NaN,NaN,NaN,NaN,0.004252,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,187105,4.86,0.2600,0.4000,NaN,NaN,NaN,NaN,NaN,NaN,0.004643,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,202108,4522.68,58.7913,169.8333,0.184756,0.0005,0.0255,0.0324,0.0128,0.014846,0.000000,0.002066,-0.0035,-0.0045,0.000602,NaN,0.030600,0.029205
1808,202109,4307.54,59.2545,175.3700,0.193036,0.0004,0.0253,0.0323,0.0137,0.015598,0.000000,0.002716,-0.0250,-0.0194,0.001393,NaN,-0.046076,-0.047152
1809,202110,4605.38,59.6354,182.8600,0.182389,0.0005,0.0268,0.0335,0.0158,0.013368,0.000000,0.008308,0.0051,0.0159,0.001151,NaN,0.070510,0.069627
1810,202111,4567.00,60.0162,190.3500,0.189455,0.0005,0.0262,0.0328,0.0156,0.015640,0.000100,0.004913,0.0210,0.0094,0.001327,NaN,-0.007256,-0.008665


### Calculating Predictors

Note: we lag EQPREM by 1 for the sake of easier computation of forecast models

In [3]:
data_stocks = data.copy()[['yyyymm']]

data_stocks['EQPREM'] = np.log(1 + data.loc[:,'CRSP_SPvw'].shift(-1)) - np.log(1 + data.loc[:,'Rfree'])

data_stocks['DP'] = np.log(data['D12'] / data['Index'])

data_stocks['DY'] = np.log(data['D12'] / data['Index'].shift(1))

data_stocks['DE'] = np.log(data['D12'] / data['E12'])

data_stocks['EP'] = np.log(data['E12'] / data['Index'])

data_stocks['SVAR'] = data['svar']

data_stocks['BM'] = data['b/m']

data_stocks['NTIS'] = data['ntis']

data_stocks['TBL'] = data['tbl']

data_stocks['LTY'] = data['lty']

data_stocks['LTR'] = data['ltr']

data_stocks['TMS'] = data['lty'] - data['tbl']

data_stocks['DFY'] = data['BAA'] - data['AAA']

data_stocks['DFR'] = data['corpr'] - data['ltr']

data_stocks['INFL'] = data['infl']

data_stocks = data_stocks.drop(columns=['LTY', 'DE'])

data_stocks.iloc[IND_TRAIN_START:IND_TEST_END,:].to_csv('data_stocks.csv', index=False)

data_stocks

,yyyymm,EQPREM,DP,DY,EP,SVAR,BM,NTIS,TBL,LTR,TMS,DFY,DFR,INFL
0,187101,NaN,-2.837728,NaN,-2.406945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,187102,NaN,-2.851151,-2.837728,-2.420368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,187103,NaN,-2.875302,-2.851151,-2.444519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,187104,NaN,-2.903111,-2.875302,-2.472328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,187105,NaN,-2.928112,-2.903111,-2.497329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,202108,-0.047171,-4.342866,-4.314288,-3.282043,0.000602,0.184756,0.014846,0.0005,-0.0035,0.0123,0.0069,-0.0010,0.002066
1808,202109,0.068135,-4.286281,-4.335018,-3.201224,0.001393,0.193036,0.015598,0.0004,-0.0250,0.0133,0.0070,0.0056,0.002716
1809,202110,-0.007282,-4.346731,-4.279873,-3.226260,0.001151,0.182389,0.013368,0.0005,0.0051,0.0153,0.0067,0.0108,0.008308
1810,202111,0.042466,-4.331997,-4.340366,-3.177747,0.001327,0.189455,0.015640,0.0005,0.0210,0.0151,0.0066,-0.0116,0.004913


In [8]:
data[['CRSP_SPvw']].shift(-1).iloc[IND_TRAIN_START:IND_TEST_END,:].to_csv('data_return.csv', index=False)

In [15]:
data_stocks.iloc[IND_TRAIN_START:IND_TRAIN_END,:]

,yyyymm,EQPREM,DP,DY,EP,SVAR,BM,NTIS,TBL,LTR,TMS,DFY,DFR,INFL
671,192612,-0.005710,-2.973012,-2.956570,-2.386837,0.000465,0.441476,0.050876,0.0307,0.0078,0.0047,0.0100,-0.0022,0.000000
672,192701,0.042019,-2.942374,-2.963349,-2.374773,0.000470,0.443706,0.050824,0.0323,0.0075,0.0028,0.0095,-0.0019,-0.011299
673,192702,0.004701,-2.979535,-2.932946,-2.430353,0.000287,0.428501,0.051668,0.0329,0.0088,0.0018,0.0092,-0.0019,-0.005714
674,192703,0.009941,-2.976535,-2.970053,-2.445079,0.000924,0.469765,0.046357,0.0320,0.0253,0.0011,0.0092,-0.0170,-0.005747
675,192704,0.057989,-2.984225,-2.967143,-2.471309,0.000603,0.456754,0.050514,0.0339,-0.0005,-0.0006,0.0090,0.0060,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026,195607,-0.035535,-3.304600,-3.254361,-2.631871,0.000496,0.524903,0.024754,0.0231,-0.0209,0.0082,0.0052,0.0116,0.007353
1027,195608,-0.046719,-3.258429,-3.297237,-2.606265,0.001235,0.541391,0.023811,0.0260,-0.0187,0.0065,0.0050,-0.0021,-0.003650
1028,195609,0.004799,-3.204645,-3.251175,-2.573142,0.000788,0.571910,0.029362,0.0284,0.0050,0.0040,0.0051,-0.0038,0.003663
1029,195610,-0.004420,-3.227967,-3.222908,-2.583039,0.001642,0.566427,0.029457,0.0290,-0.0054,0.0039,0.0058,-0.0051,0.003650


In [16]:
data_stocks.iloc[IND_TEST_START:IND_TEST_END,:]

,yyyymm,EQPREM,DP,DY,EP,SVAR,BM,NTIS,TBL,LTR,TMS,DFY,DFR,INFL
1032,195701,-0.027931,-3.248434,-3.291115,-2.574677,0.000902,0.567243,0.027992,0.0311,0.0346,0.0017,0.0072,-0.0149,0.000000
1033,195702,0.021150,-3.217201,-3.250394,-2.542483,0.001056,0.584994,0.030173,0.0310,0.0025,0.0018,0.0080,0.0068,0.003623
1034,195703,0.043190,-3.238565,-3.219107,-2.562911,0.000330,0.599819,0.026600,0.0308,-0.0024,0.0023,0.0077,0.0074,0.003610
1035,195704,0.035506,-3.274852,-3.238565,-2.597229,0.000302,0.576098,0.027421,0.0307,-0.0222,0.0038,0.0077,0.0156,0.003597
1036,195705,-0.003302,-3.311134,-3.274852,-2.631575,0.000482,0.564039,0.028849,0.0306,-0.0023,0.0042,0.0078,-0.0052,0.003584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1746,201607,0.001046,-3.885319,-3.850329,-3.210865,0.000478,0.314661,-0.031675,0.0030,0.0081,0.0145,0.0094,0.0164,-0.001618
1747,201608,0.000246,-3.879884,-3.881104,-3.201425,0.000279,0.315197,-0.030782,0.0030,-0.0140,0.0156,0.0092,0.0156,0.000918
1748,201609,-0.018321,-3.874451,-3.875687,-3.192038,0.001673,0.316794,-0.032603,0.0029,-0.0124,0.0167,0.0090,0.0005,0.002404
1749,201610,0.034964,-3.849850,-3.869467,-3.152198,0.000364,0.319688,-0.029034,0.0033,-0.0314,0.0187,0.0087,0.0051,0.001247
